# Project 2: Gauss Elimination and Least Square

(1) Write your name here: Xuemei Chen

## Computing power
The LINPACK Benchmarks are a measure of a system's floating point computing power. Introduced by Jack Dongarra, they measure how fast a computer solves a dense n by n system of linear equations $Ax = b$. This is the standard measure for computer speed. Supercomputers (which usually have linux operating systems) can do about 10 petaflops in one second. One petaflop is $10^{15}$ flops. 

Titan, a supercomputer in the Oak Ridge National lab, is the fastest computer in the US. It can perform 17.59 petaflops every second.

(2) Calculate how much time (in seconds) does Titan need to solve a 5000 by 5000 system $Ax=b$, assuming it takes $\frac{2}{3}n^3$ flops for Gauss Elimination?

In [6]:
(5000**3*2/3.0)/(17.59*10**15)

4.737540269092287e-06

Laptops nowadays is around 10's gigaflops. **Now let's see how fast your own machine is**.

(3) Create a 5000 by 5000 random matrix $R$, and create a 5000 by 1 random vector $rb$, where each entry of $R$ and $rb$ is standard normal distribution (Hint: look up numpy.random)

In [1]:
import numpy as np
R = np.random.randn(5000,5000)
rb = np.random.randn(5000,1)

(4) Measure how much time does it take to solve this random $Rx=rb$. Below is an easy way to measure time

```python
import timeit
start = timeit.default_timer()

#Your operation here

end = timeit.default_timer()
print end - start
```

In [64]:
import timeit
from scipy.linalg import solve,inv
start = timeit.default_timer()
solve(R,rb)
end = timeit.default_timer()
et = end - start
print et

2.34996700287


(5) Now calculate how many gegaflops your computer can do in a second.

In [4]:
6000**3*2/3.0/et/10**9

58.018639015824924

(5) Also measure how much total time your computer takes to find inverse of $R$ and multiply $R^{-1}*b$. You should get the message that this is a worse way to solve $Rx=rb$. In general, it never does you any good to find inverse in numerical computing.

In [5]:
start = timeit.default_timer()
Ri = inv(R)
x = np.dot(Ri,rb)
end = timeit.default_timer()
et2 = end - start
print et2

4.15607619286


## Gaussian Elimination from scratch

Below is the SPLU function that I wrote. It decompose $A$ as $A=PLU$ using **Scaled Partial Pivoting**, where $P$ is a permutation matrix, $L$ is lower triangular, and $U$ is upper triangular.

In [1]:
import numpy as np
# Input A is any invertible matrix
# return P,L,U such that A = PLU
def SPLU(A):     
    sA = A.shape
    # make sure it is in float
    A = A.astype(float)
    
    #Check A is square    
    if sA[0] != sA[1]:
        raise ValueError('You need to enter a square matrix.')
    n = sA[0]    
    # find the max in each row
    AA = np.absolute(A)
    s = np.amax(AA,axis=1)
    RowInd = []  #Initiate an empty list for row indices
    L = np.identity(n)
    for j in range(n-1): # at column j
        # Find the row index of the pivot
        allRI = list(set(range(n)) - set(RowInd))
        allRelV = AA[allRI,j]/s[allRI]
        fi = np.argmax(allRelV)
        ri = allRI[fi] #ri is the pivot row idex        
        #update RowInd
        RowInd.append(ri)        
        for i in list(set(allRI) - set([ri])): 
            if A[ri,j] == 0:
                raise ValueError('You need to enter an invertible matrix.')
            c = -A[i,j]/A[ri,j]
            A[i,j:n] = A[ri, j:n]*c + A[i,j:n] 
            if c != 0:
                L[i,ri]= -c
        print A
    # build P,L,U according to RowInd
    LastL = list(set(range(n)) - set(RowInd))
    LastI = LastL[0]
    RowInd.append(LastI)
    I = np.identity(n)
    P = I[:,RowInd]
    U = A[RowInd,:]
    L = L[:,RowInd] #permute columns 
    L = L[RowInd,:]
    return (P, L, U)

(6) Use the above function to check your work on chapter 4 exercise 6. Print out P,L,U, and make sure their product is the original matrix.

In [2]:
B = np.array([[3,6,-3],[2,0,6],[-4,7,4]])
(P,L,U) = SPLU(B)
print U

[[  0.    11.25   0.  ]
 [  0.     3.5    8.  ]
 [ -4.     7.     4.  ]]
[[  0.    11.25   0.  ]
 [  0.     0.     8.  ]
 [ -4.     7.     4.  ]]
[[ -4.     7.     4.  ]
 [  0.    11.25   0.  ]
 [  0.     0.     8.  ]]


(7) Write a function SL(L,b) to solve $Lx=b$ where $L$ is lower triangular. Make sure you first verify that $L$ is lower triangular and invertible. 
## Try to write comments/explanations when you write a function

In [3]:
def SL(L,b):
    L = L.astype(float)
    sL = L.shape
    if sL[0]!= sL[1]:
        raise ValueError('You need to enter a square lower triangular matrix.')
    n = sL[0]
    for i in range(n):
        for j in range(i+1,n):
            if L[i][j] !=0:
                raise ValueError('You need to enter a square lower triangular matrix.')
    
    # Initiate x
    x = np.zeros(n)
    for i in range(n): #solve forwardly
        if L[i,i] == 0:
            raise ValueError('Your L is not invertible.')
        x[i] = (b[i] - np.sum(x[:i]*L[i,:i]))/L[i,i]
    return x

(8) Test your SL function on chapter 4 exercise 5, where L is the lower triangular matrix there, and b=(-3,-22,3).

**Call the solution y**

In [6]:
L = np.array([[3,0,0],[2,4,0],[-4,-1,2]])
bb = np.array([-3,-22,3])
y = SL(L,bb)
print y
yy = np.linalg.solve(L,bb)
print yy

[-1. -5. -3.]
[-1. -5. -3.]


(9) Write a function SU(U,b) to solve $Ux=b$ where $U$ is upper triangular. Make sure you verify that $U$ is upper triangular and invertible. This should be only a small modification of your previous function.

In [7]:
def SU(U,b):
    U = U.astype(float)
    sU = U.shape
    if sU[0]!= sU[1]:
        raise ValueError('You need to enter a square upper triangular matrix.')
    n = sU[0]
    for i in range(n):
        for j in range(i):
            if U[i][j] !=0:
                raise ValueError('You need to enter a square upper triangular matrix.')
    
    # Initiate x
    x = np.zeros(n)
    for i in range(n-1,-1,-1): #solve backwards
        if U[i,i] == 0:
            raise ValueError('Your U is not invertible.')
        x[i] = (b[i] - np.sum(x[i+1:]*U[i,i+1:]))/U[i,i]
    return x

(10) Test your SU function on chapter 4 exercise 5, where U is the upper triangular matrix there, and b is the y vector that you solved in (8).

In [15]:
U = np.array([[1,-2,-1],[0,1,2],[0,0,1]])
x = SU(U,y)
print x
xx = np.linalg.solve(U,y)
print xx

[-2.  1. -3.]
[-2.  1. -3.]


(11) Write a function MySolve(A,b) that solves Ax=b using scaled partial pivoting. Your code will call SPLU, SU, SL, and should be only a few lines.

In [10]:
import numpy as np
import scipy.linalg as sl
def MySolve(A,b):
    (P,L,U) = SPLU(A)
    #(P,L,U) = sl.lu(A)
    #need to solve PLUx=b, which is LUx = P'b
    newb = np.dot(np.transpose(P),b)
    y = SL(L,newb)
    x = SU(U,y)
    return x

(12) Test MySolve on chapter 4 exercise 5.

In [11]:
A = np.array([[3,-6,-3],[2,0,6],[-4,7,4]])
b=np.array([-3,-22,3])
#x = MySolve(A,b)
x= np.linalg.solve(A,b)
print x
#newx = np.array([-2,1,-3])
print np.dot(A,np.array([-2,1,-3]))


[-2.  1. -3.]
[ -3 -22   3]


## Least Square via Cholesky

(13) Write a function LS(A,b) to find the least square solution of Ax=b, where A has independent columns. Your algoritm will first use python's cholesky function, then use your own SL and SU functions to find the final solution.

In [12]:
import numpy as np
import numpy.linalg as nl
def LS(A,b):
    At = np.transpose(A)
    B = np.dot(At,A)
    L = nl.cholesky(B)
    y = SL(L,np.dot(At,b))
    U = np.transpose(L)
    x = SU(U,y)
    return x

Below is the scores of games played between Team A, B, C, D, E

| Home | Away | Home score | Away score |
|:--- | :---  | :---  | :---  |
|A|B|10|4|
|A|C|1|3|
|A|D|8|4|
|A|E|6|4|
|B|A|5|9|
|B|C|1|13|
|B|D|5|4|
|C|A|16|12|
|C|B|11|13|
|C|D|8|4|
|C|E|9|3|
|D|A|8|13|
|D|B|5|4|
|D|C|6|12|
|E|B|3|4|
|E|C|1|3|
|E|D|3|6|

(14) Extra credit problem: Try your best to rank these 5 teams simply by staring at the table:

(15) Assign each team an overall score $s_i$ and use the function you wrote in (13) to find the scores. We can assume $\sum s_i=100$.

In [13]:
G = np.zeros((17,5))
G = np.array([[1,-1,0,0,0],[1,0,-1,0,0],[1,0,0,-1,0],[1,0,0,0,-1],[-1,1,0,0,0],[0,1,-1,0,0],[0,1,0,-1,0]])
G = np.vstack([G,[-1,0,1,0,0],[0,-1,1,0,0],[0,0,1,-1,0],[0,0,1,0,-1]])
G = np.vstack([G,[-1,0,0,1,0],[0,-1,0,1,0],[0,0,-1,1,0],[0,-1,0,0,1],[0,0,-1,0,1],[0,0,0,-1,1],[1,1,1,1,1]])
b = np.array([6,-2,4,2,-4,-12,1,4,-2,4,6,-5,1,-6,-1,-2,-3,100])

In [14]:
LS(G,b)

array([ 21.86296296,  18.08518519,  23.4       ,  18.41851852,  18.23333333])

(16) Rank these 5 teams according to your solution:

C>A>D>E>B